# Fabric Analytics Roadshow - Deployment Notebook

This notebook deploys all lab assets into your Fabric workspace.

## Prerequisites
- Access to a Microsoft Fabric workspace
- Contributor or Admin permissions in the workspace

## What This Notebook Does
1. Clones Analytics Roadshow GIT Repo to a temp directory
1. Install Pilot version of fabric-cicd library that supports Spark Job Definitions
1. Uses fabric-cicd python library to deploy the project to the target workspace

In [ ]:
# Define the Workspace ID that the solution will be deployed into. This must be a Fabric Capacity enabled workspace.
workspace_id = "1e5cfd3c-ef5a-4cc1-8e22-2b3ed092c779"

In [11]:
import tempfile
import subprocess
import os

# Create a temporary directory
temp_dir = tempfile.mkdtemp(prefix="fabric_roadshow_")
print(f"Created temporary directory: {temp_dir}")

# Clone the specific branch
repo_url = "https://github.com/microsoft/fabric-analytics-roadshow-lab.git"
branch_name = "initial-version-prep"

print(f"Cloning {repo_url} (branch: {branch_name})...")
result = subprocess.run(
    ["git", "clone", "--branch", branch_name, "--single-branch", repo_url, temp_dir],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print(f"✅ Successfully cloned repository to: {temp_dir}")
    workspace_root = os.path.join(temp_dir, "workspace")
    print(f"📁 Workspace directory: {workspace_root}")
else:
    print(f"❌ Error cloning repository:")
    print(result.stderr)
    raise Exception("Failed to clone repository")

Created temporary directory: C:\Users\MILESC~1\AppData\Local\Temp\fabric_roadshow_vie1nfnv
Cloning https://github.com/microsoft/fabric-analytics-roadshow-lab.git (branch: initial-version-prep)...
✅ Successfully cloned repository to: C:\Users\MILESC~1\AppData\Local\Temp\fabric_roadshow_vie1nfnv
📁 Workspace directory: C:\Users\MILESC~1\AppData\Local\Temp\fabric_roadshow_vie1nfnv\workspace


In [12]:
# Install fabric_cicd from the cloned repository
whl_path = os.path.join(temp_dir, "setup", "fabric_cicd-0.1.33-py3-none-any.whl")
print(f"Installing fabric_cicd from: {whl_path}")

import subprocess
import sys

result = subprocess.run(
    [sys.executable, "-m", "pip", "install", whl_path, "--force-reinstall"],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✅ Successfully installed fabric_cicd")
else:
    print(f"❌ Installation failed:")
    print(result.stderr)

Installing fabric_cicd from: C:\Users\MILESC~1\AppData\Local\Temp\fabric_roadshow_vie1nfnv\setup\fabric_cicd-0.1.33-py3-none-any.whl
✅ Successfully installed fabric_cicd


In [ ]:
from fabric_cicd import FabricWorkspace, publish_all_items, unpublish_all_orphan_items, append_feature_flag

from fabric_cicd import change_log_level
#change_log_level('DEBUG')
append_feature_flag("enable_lakehouse_unpublish")

# Use the workspace directory from the cloned repository
repository_directory = workspace_root

item_type_in_scope = ["Lakehouse", "Warehouse", "Notebook", "SparkJobDefinition", "Lakehouse", "Environment"]  # "" , "SparkJobDefinition", "Lakehouse", "Environment", 

# Initialize the FabricWorkspace object with the required parameters
target_workspace = FabricWorkspace(
    workspace_id=workspace_id,
    repository_directory=repository_directory,
    item_type_in_scope=item_type_in_scope
)

# Publish all items defined in item_type_in_scope
publish_all_items(target_workspace)